In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import plotly.express as px

In [2]:
data = pd.read_excel("/Users/zihuiouyang/Documents/CO2_H2O_project/Data/Solubility_database5-12.xlsx", header=1)
clean_data = data[data["Phases"] == "liq"]
clean_data1 = data[data["Phases"] == "liq+fl"]
clean_data2 = data[data["Phases"].isnull()]
data = pd.concat([clean_data, clean_data1, clean_data2], ignore_index = True)
data = data[data["PH2O (bar)"].notna()]
data = data[data["H2O Glass (wt.%)"].notna()]
data = data.to_numpy()
copy_of_data = data

In [3]:
copy_of_data = copy_of_data[copy_of_data[:,46]!=0]
#Data used for fitting the model
idx = [9,11,46,118,162,163,164,165,166,167,168,169,170,171,0,46]
reduced_data = copy_of_data[:,idx]

idx1 = []
for i in range(np.shape(reduced_data)[0]):
    if (2*reduced_data[i,4]+2*reduced_data[i,5]+3*reduced_data[i,6]+reduced_data[i,7]+reduced_data[i,9]+reduced_data[i,10]+reduced_data[i,11]+reduced_data[i,12]) != 0:
        idx1.append(i)

reduced_data = reduced_data[idx1]
reduced_data = reduced_data[reduced_data[:,3]!=0]

idx2 = []
for i in range(np.shape(reduced_data)[0]):
    if type(reduced_data[i,0]) is str:
        idx2.append(i)
reduced_data = np.delete(reduced_data, idx2, 0)

idx3 = []
for i in range(np.shape(reduced_data)[0]):
    if type(reduced_data[i,1]) is str:
        idx3.append(i)
reduced_data = np.delete(reduced_data, idx3, 0)
idx1 = []
for i in range(np.shape(reduced_data)[0]):
    if reduced_data[i,10]+reduced_data[i,11]+reduced_data[i,12] != 0:
        idx1.append(i)

reduced_data = reduced_data[idx1]
new_train = np.ones((np.shape(reduced_data)[0],5))
new_train[:,1] = np.log(reduced_data[:,2].astype("float"))
NBO = 2*(reduced_data[:,7]+reduced_data[:,9]+reduced_data[:,10]+reduced_data[:,11]+reduced_data[:,12]-reduced_data[:,6])
NBOO = NBO/(2*reduced_data[:,4]+2*reduced_data[:,5]+3*reduced_data[:,6]+reduced_data[:,7]+reduced_data[:,9]+reduced_data[:,10]+reduced_data[:,11]+reduced_data[:,12])
new_train[:,2] = NBOO

a  = reduced_data[:,0].astype("float")
b = reduced_data[:,1].astype("float")
new_train[:,3] = a/b
new_train[:,4] = 1/reduced_data[:,1]

y_whole_set = np.log(reduced_data[:,3].astype("float"))
new_train1 = np.append(new_train, np.reshape(reduced_data[:,14],(-1,1)),1)
new_train1 = np.append(new_train1, np.reshape(reduced_data[:,15],(-1,1)),1)
#Train-validation-test split: 60-20-20, using ordinary linear regression
X_train, X_test, y_train, y_test = train_test_split(new_train1, y_whole_set, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) 

In [8]:
y_test = np.exp(y_whole_set)
a2 = np.reshape((new_train1[:,6].astype("float64")), (-1,1))

In [9]:
df = np.append(a2, np.reshape(y_test, (-1,1)),1)
df = np.append(df, np.reshape(new_train1[:,5], (-1,1)),1)
df = pd.DataFrame(df, columns = ["logPH2O", "ground_truth_H2O", "experiments"])

In [10]:
fig = px.scatter(df, x="logPH2O", y="ground_truth_H2O", color="experiments")
fig.show()